In [4]:
import json
import random
import torch
import time
import sys
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Load the model
model = tf.keras.models.load_model('medical_chatbot_model.h5')

# Load the TfidfVectorizer
with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

# Load the label encoder
with open('label_encoder.pkl', 'rb') as f:
    le = pickle.load(f)

In [8]:
# Load the disease data
with open("data.json", "r") as f:
    disease_data = json.load(f)

# Initialize lists to hold our sentences (intents), labels (diseases), and responses
sentences = []
labels = []
descriptions = []
treatments = []
# Fill lists with data from JSON
for intent in disease_data["intents"]:
    for pattern in intent["symptoms"]:
        sentences.append(pattern)
        labels.append(intent["name"])
        descriptions.append(intent["description"])
        treatments.append(intent["treatments"])
        
# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Stop words set
stop_words = set(stopwords.words('english'))

In [14]:
def greet_user():
    greetings = ["Hi! How can I assist you today?", "Hello! How can I help you today?"]
    return random.choice(greetings)

def get_treatment(user_input):
    if "treatment" in user_input.lower() or "cure" in user_input.lower() or "treatments" in user_input.lower() or "cures" in user_input.lower() or "what are the treatments" in user_input.lower():
        return True
    else:
        return False

def possible_disease_enquiry(user_input):
    enquiry_words = ["possible disease","possible diseases","chances","chance","predicted disease","predicted illness","scope","predict"]
    return any(word in user_input.lower() for word in enquiry_words)

def is_exit_message(user_input):
    exit_words = ["thank", "thanks", "goodbye", "bye", "quit"]
    return any(word in user_input.lower() for word in exit_words)

def preprocess_text(text):
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)

def extract_symptoms(input_text):
    word_list = nltk.word_tokenize(input_text)
    symptoms = [lemmatizer.lemmatize(word.lower()) for word in word_list if word not in stop_words]
    return symptoms

def predict_intent(user_input):
    treatmentss = []
    input_vector = vectorizer.transform([user_input]).toarray()
    input_vector = np.reshape(input_vector, (input_vector.shape[0], 1, input_vector.shape[1]))
    prediction = model.predict(input_vector)
    predicted_label = le.inverse_transform([np.argmax(prediction)])
    predicted_disease = predicted_label[0]
    for intent in disease_data["intents"]:
        if intent["name"].lower() == predicted_disease.lower():
            treatments = intent["treatments"]
            if treatments:
                for treatment in treatments:
                    treatmentss.append(treatment)
    for intent in disease_data["intents"]:
        if intent["name"].lower() == predicted_disease.lower():
            description = intent["description"]
    confidence_level = np.max(prediction)
    return description, treatmentss, predicted_disease, confidence_level

def get_possible_diseases(user_input):
    symptoms = extract_symptoms(user_input)
    possible_diseases = []

    for intent in data["intents"]:
        for symptom in symptoms:
            if symptom in intent["symptoms"]:
                possible_diseases.append(intent["name"])
                break

    return possible_diseases

def get_possible_diseases_and_treatments(user_input):
    symptoms = extract_symptoms(user_input)
    possible_diseases = []
    treatments = {}

    for intent in disease_data["intents"]:
        for symptom in symptoms:
            if symptom in intent["symptoms"]:
                possible_diseases.append(intent["name"])
                treatments[intent["name"]] = intent["treatments"]
                break

    return possible_diseases, treatments

def resetting_user_input(user_input):
    if "another symptoms" in user_input.lower() or "help me with another set of symptoms" in user_input.lower() or "new set of symptoms" in user_input.lower() or "fresh start" in user_input.lower() or "from beginning" in user_input.lower():
        return True
    else:
        return False

def typing_effect(message):
    for char in message:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(0.05)
    print("")   

In [20]:
typing_effect(greet_user())
symptoms_input = ""
possible_diseases = []
possible_treatments = {}
# Confidence threshold
confidence_threshold = 0.38
while True:
    # User enters a symptom
    user_input = input("User: ")
    
    if resetting_user_input(user_input):
        symptoms_input = ""
        possible_diseases = []
        possible_treatments = {}
        typing_effect("Sure, how can I help you")
        continue
       
    if is_exit_message(user_input):
        typing_effect("Chatbot: You're welcome! Take care!")
        break

    # Add symptom to user's symptoms
    symptoms_input += " " + user_input
    description, treatments, predicted_disease, confidence = predict_intent(symptoms_input)
    
    if confidence < confidence_threshold:
        if possible_disease_enquiry(user_input):
            possible_diseases, possible_treatments = get_possible_diseases_and_treatments(symptoms_input)
            if len(possible_diseases)>0:
                typing_effect(f"Chatbot: The possible diseases based on your symptoms are: {', '.join(possible_diseases)}")
            else:
                typing_effect("No other predictions as per my knowledge")
            continue    

        if get_treatment(user_input):
            if len(possible_diseases) > 0:
                for disease in possible_diseases:
                    typing_effect(f"Chatbot: The treatments for {disease} are: {', '.join(possible_treatments[disease])}")
            else:
                typing_effect("Chatbot: You haven't asked for possible diseases yet. Please ask for possible diseases first.")
            continue
            
        common_symptoms_response = ["Your symptoms are quite common, could you please provide more details or any other symptom?", 
                                   "That is a very common symptom, provide me with another symptom you are facing","That's a quite common symptom for many diseases, provide me with any another symptom you are having?"]
        sol = random.choice(common_symptoms_response)
        typing_effect(f"Chatbot: {sol}")
        typing_effect(f"Confidence: {confidence}")
        continue
        
    for item in description:
        print(*item, end="")
    print()
        
    typing_effect(f"Confidence: {confidence}")
    
    user_input = input("User: ")
    
    if get_treatment(user_input):
        typing_effect(f"Chatbot: The treatments are: {treatments}")
        continue
    
    if resetting_user_input(user_input):
        symptoms_input = ""
        possible_diseases = []
        possible_treatments = {}
        typing_effect("Sure, how can I help you")
        continue  
        
    if possible_disease_enquiry(user_input):
        possible_diseases, possible_treatments = get_possible_diseases_and_treatments(symptoms_input)
        if len(possible_diseases)>0:
            typing_effect(f"Chatbot: The possible diseases based on your symptoms are: {', '.join(possible_diseases)}")
        else:
            typing_effect("No other predictions as per my knowledge")
        continue  
        
    if is_exit_message(user_input):
        typing_effect("Chatbot: You're welcome! Take care!")
        break

Hi! How can I assist you today?
User: I have rashes
1/1 [==============================] - 0s 114ms/step
Systemic lupus erythematosus (SLE) is an autoimmune disease that can affect various parts of the body, including the skin, joints, kidneys, brain, and other organs.
Confidence: 0.998524010181427
User: What are the treatments
Chatbot: The treatments are: ['Nonsteroidal anti-inflammatory drugs (NSAIDs)', 'Antimalarial drugs', 'Corticosteroids', 'Immunosuppressants', 'Biologics']
User: thanks
Chatbot: You're welcome! Take care!
